In [2]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
conn_str = 'DRIVER={SQL Server};SERVER=AZ2D-MASKDB-220.DEVCDT.COM.BR;DATABASE=Fortbrasil;Integrated Security=true'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [16]:
codigo = """
SELECT a.name FROM SYS.DATABASES AS A
WHERE
A.NAME NOT IN ('master','tempdb','model','msdb','DBA')
"""
cursor.execute(codigo)

In [17]:
emissores = []
emissores.append(cursor.fetchall())

In [60]:
codigo = """

SET NOCOUNT ON 
IF OBJECT_ID('tempdb..#TRANSACOESIOF') IS NOT NULL
BEGIN
    DROP TABLE #TRANSACOESIOF
END
SELECT TT.Nome,TR.Id_Conta,TR.DatavencimentoPadrao,TR.DataGeracao,SUM(TR.Valor) AS Valor
INTO #TRANSACOESIOF
FROM TransacoesCorrentes AS TR WITH(NOLOCK)
INNER JOIN Tipostransacoes AS TT WITH(NOLOCK) ON (TT.Id_TipoTransacao = TR.Id_TipoTransacao)
WHERE
TR.Id_Tipotransacao = 117 AND
TR.DataGeracao BETWEEN '2018-01-01' AND '2018-12-31'
GROUP BY TT.Nome,TR.Id_Conta,TR.DatavencimentoPadrao,TR.DataGeracao

CREATE CLUSTERED INDEX IDX_TMP_CONTAS ON #TRANSACOESIOF(Id_Conta)

SELECT A.Nome,A.Id_Conta,AVG(A.Valor) AS Media FROM #TRANSACOESIOF AS A
GROUP BY A.Nome,A.Id_Conta

"""
cursor.execute(codigo)
dataset = []
dataset.append(cursor.fetchall())

In [86]:
%matplotlib inline
transacoes = [x[0] for x in dataset[0]]
contas = [x[1] for x in dataset[0]]
valores = [x[2] for x in dataset[0]]
df = pd.DataFrame({'Transacao':transacoes,
                  'Id_Conta':contas,
                  'Valores':valores})

In [1]:
df

NameError: name 'df' is not defined